In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
#from sklearn.metrics import mean_squared_errosr
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
results_dir_priors = "results/priors/single_layer/tanh/friedman"
results_dir_posteriors = "results/regression/single_layer/tanh/friedman"
#results_dir_posteriors_dst = "results/regression/single_layer/tanh/friedman/full_regularization"

prior_names_and_configs = {"Gaussian": "gauss", "Regularized Horseshoe": "reg_hs", "Dirichlet Gamma": "dir_gam", "Dirichlet Horseshoe": "dir_hs", "Dirichlet Student T": "dir_stud_t"}
posterior_names = ["Dirichlet Horseshoe tanh", "Regularized Horseshoe tanh", "Dirichlet Student T tanh", "Gaussian tanh"]
#posterior_names_and_configs_dst = ["Dirichlet Student T tanh"]
   
prior_fits = {}
posterior_fits = {}

for key, value in prior_names_and_configs.items():
    prior_fit = get_model_fits(
        config=value,
        results_dir=results_dir_priors,
        models=key,
        include_prior=False,
    )
    prior_fits[key] = prior_fit

prior_fits = {outer: inner[outer] for outer, inner in prior_fits.items()}
    
posterior_N100_fits = get_model_fits(
    config="Friedman_N100_p10_sigma1.00_seed1",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)

posterior_N200_fits = get_model_fits(
    config="Friedman_N200_p10_sigma1.00_seed2",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)

posterior_N500_fits = get_model_fits(
    config="Friedman_N500_p10_sigma1.00_seed11",
    results_dir=results_dir_posteriors,
    models=posterior_names,
    include_prior=False,
)


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpmath import hyper
from mpmath import gamma
from scipy.special import poch


p=10
alpha=1.0
beta=(p-1)*alpha


def p_kappa_dirichlet_horseshoe(kappa, alpha, beta, a_j=1.0):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    prefactor = (1/np.pi) * (a_j / ((1-kappa) * np.sqrt(kappa) * np.sqrt(1-kappa)))# * (1/p)
    c = (kappa / (1-kappa))*(a_j**2)
    # {}_3F_2([1, 1.1/2, 2.1/1], [1, 3/2], z)
    gamma_const = (gamma(alpha+1/2)/gamma(alpha))*(gamma(alpha+beta)/(gamma(alpha+beta+1/2)))
    hyper_val = hyper([1, alpha+1/2], [alpha+beta+1/2], -c)
    return float(prefactor * gamma_const * hyper_val)


def p_kappa_dirichlet_student_t(kappa, alpha, beta, nu=3.0, a_j=1.0):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    C = gamma((nu+1)/2) * 1/(np.sqrt(np.pi * nu) * gamma(nu/2))
    prefactor = ((a_j**nu) * nu**((nu+1)/2) * kappa**(nu/2 - 1)) / ((1-kappa)**(nu/2 + 1)) # * poch(alpha, nu) / poch(p*alpha, nu)
    c = (-kappa / (1-kappa))*(a_j**2)
    gamma_const = (gamma(alpha+nu/2)/gamma(alpha))*(gamma(alpha+beta)/(gamma(alpha+beta+nu/2)))
    hyper_val = hyper([(nu+1)/2, alpha + nu/2], [alpha+beta+nu/2], c)
    return float(C * prefactor * gamma_const * hyper_val)



def p_kappa_horseshoe(kappa, a_j=1.0):
    if kappa <= 0 or kappa >= 1:
        return 0.0
    prefactor = (1/np.pi) * (a_j / ((a_j**2-1)*kappa + 1)) * 1/(np.sqrt(kappa) * np.sqrt(1-kappa))
    return float(prefactor)


I have updated the density plot!

\begin{equation}
\begin{aligned}
    \frac{1}{\pi} \frac{\sqrt{q_{j}} \tau}{(1-\kappa_j)\sqrt{\kappa_j}\sqrt{1-\kappa_j}} \frac{\Gamma(\alpha+\frac{1}{2})}{\Gamma(\alpha)}\frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha+\beta+\frac{1}{2})}{}_2F_1\!\left(\begin{matrix} 1, \alpha+\frac{1}{2} \\ \alpha + \beta + \frac{1}{2} \end{matrix}; -c \right)
\end{aligned} 
\end{equation}

\begin{equation}
\begin{aligned}
\frac{\Gamma(\frac{\nu+1}{2})}{\sqrt{\nu\pi}\Gamma(\frac{\nu}{2})}  \frac{1}{(1-\kappa)^{\frac{\nu}{2}+1}}\nu^{\frac{\nu + 1}{2}} \kappa^{\frac{\nu}{2}-1}q_{j}^{\frac{\nu}{2}}\tau^{\nu} \mathbb{E}_{\xi_j}\left[\frac{\xi_j^{\nu/2}}{\left(1+c\xi_j\right)^{\frac{\nu+1}{2}}}\right]
\end{aligned} 
\end{equation}

In [4]:
# Lag kappa-grid
kappa_vals = np.linspace(0.001, 0.999, 500)
p_vals_dirichlet_horseshoe = [p_kappa_dirichlet_horseshoe(k, alpha, beta) for k in kappa_vals]
p_vals_dirichlet_horseshoe_medium_low = [p_kappa_dirichlet_horseshoe(k, alpha, beta, a_j=0.7) for k in kappa_vals]
p_vals_dirichlet_horseshoe_medium_high = [p_kappa_dirichlet_horseshoe(k, alpha, beta, a_j=2) for k in kappa_vals]
p_vals_dirichlet_horseshoe_low = [p_kappa_dirichlet_horseshoe(k, alpha, beta, a_j=0.1) for k in kappa_vals]
p_vals_dirichlet_horseshoe_high = [p_kappa_dirichlet_horseshoe(k, alpha, beta, a_j=5) for k in kappa_vals]

p_vals_dirichlet_student_t = [p_kappa_dirichlet_student_t(k, alpha, beta) for k in kappa_vals]
p_vals_dirichlet_student_t_medium_low = [p_kappa_dirichlet_student_t(k, alpha, beta, a_j=0.7) for k in kappa_vals]
p_vals_dirichlet_student_t_medium_high = [p_kappa_dirichlet_student_t(k, alpha, beta, a_j=2) for k in kappa_vals]
p_vals_dirichlet_student_t_low = [p_kappa_dirichlet_student_t(k, alpha, beta, a_j=0.1) for k in kappa_vals]
p_vals_dirichlet_student_t_high = [p_kappa_dirichlet_student_t(k, alpha, beta, a_j=5) for k in kappa_vals]

p_vals_horseshoe = [p_kappa_horseshoe(k) for k in kappa_vals]
p_vals_horseshoe_medium_low = [p_kappa_horseshoe(k, a_j=0.7) for k in kappa_vals]
p_vals_horseshoe_medium_high = [p_kappa_horseshoe(k, a_j=2) for k in kappa_vals]
p_vals_horseshoe_low = [p_kappa_horseshoe(k, a_j=0.1) for k in kappa_vals]
p_vals_horseshoe_high = [p_kappa_horseshoe(k, a_j=5) for k in kappa_vals]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 6), sharex=True, sharey=False)

axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_low)#, label=r"$a_j = 0.1$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_medium_low)#, label=r"$a_j = 0.7$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe)#, label=r"$a_j = 1$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_medium_high)#, label=r"$a_j = 2$")
axes[0].plot(kappa_vals, p_vals_dirichlet_horseshoe_high)#, label=r"$a_j = 5$")
axes[0].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[0].set_xlabel(r"$\kappa$")
axes[0].set_title("Dirichlet–Horseshoe")
axes[0].set_ylim((0, 7))
axes[0].legend()

axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_low, label=r"$a_j = 0.1$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_medium_low, label=r"$a_j = 0.7$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t, label=r"$a_j = 1$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_medium_high, label=r"$a_j = 2$")
axes[1].plot(kappa_vals, p_vals_dirichlet_student_t_high, label=r"$a_j = 5$")
#axes[1].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[1].set_xlabel(r"$\kappa$")
axes[1].set_title("Dirichlet–Student T")
axes[1].set_ylim((0, 7))
axes[1].legend(loc='upper center')

axes[2].plot(kappa_vals, p_vals_horseshoe_low)#, label=r"$a_j = 0.1$")
axes[2].plot(kappa_vals, p_vals_horseshoe_medium_low)#, label=r"$a_j = 0.7$")
axes[2].plot(kappa_vals, p_vals_horseshoe)#, label=r"$a_j = 1$")
axes[2].plot(kappa_vals, p_vals_horseshoe_medium_high)#, label=r"$a_j = 2$")
axes[2].plot(kappa_vals, p_vals_horseshoe_high)#, label=r"$a_j = 5$")
#axes[2].set_ylabel(r"$p(\kappa \mid \sigma, \tau)$")
axes[2].set_xlabel(r"$\kappa$")
axes[2].set_title("Horseshoe")
axes[2].set_ylim((0, 7))
axes[2].legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np, mpmath as mp

def poch(x, n):  # rising factorial (x)_n for real n
    return mp.gamma(x + n) / mp.gamma(x)

def theory_beta_sqrt(alpha, beta, s):
    t1 = mp.hyp2f1(1, alpha, alpha + beta, s**2)
    t2 = s * poch(alpha, 0.5) / poch(alpha + beta, 0.5) * mp.hyp2f1(1, alpha + 0.5, alpha + beta + 0.5, s**2)
    return t1 - t2

def empirical_estimate(alpha, beta, s, n=100_000, seed=123):
    rng = np.random.default_rng(seed)
    xi = rng.beta(alpha, beta, size=n)
    vals = 1.0 / (1.0 + s * np.sqrt(xi))
    m = float(vals.mean())
    se = float(vals.std(ddof=1) / np.sqrt(n))
    return m, se

# --- set parameters ---
p=10
alpha, beta = 0.1, (p-1)*alpha
s, n, seed  = 1.0, 100_000, 123  # require s > -1
# ----------------------

theory = float(theory_beta_sqrt(alpha, beta, s))
emp, se = empirical_estimate(alpha, beta, s, n, seed)
diff, z = emp - theory, (emp - theory) / se


print("theory", np.round(theory, 5), "\nempirical", np.round(emp, 5), "\nabs_diff", np.round(diff, 5), "\nSE_MC", np.round(se, 5), "\nz_score", np.round(z, 5))



In [32]:
import numpy as np, mpmath as mp
from scipy.stats import betaprime
from mpmath import hyper

n = 1_00
p=10
alpha, beta = 0.1, (p-1)*alpha
nu=3
s=0.1
t = betaprime.rvs(1/2, nu/2, size=n)
hyper_vec = np.zeros(n)
for i in range(n):
    hyper_vec[i] = hyper([1, alpha], [alpha + beta], -s*nu*t[i])

emp_mean = np.mean(hyper_vec)

theory = mp.hyper([1, alpha, 1/2], [alpha + beta, (nu+1)/2], -s*nu)
print(emp_mean-theory)

In [ ]:
import numpy as np, mpmath as mp
from scipy.stats import betaprime

# Parametre
n = 100
p = 10
alpha = 0.1
beta = (p-1)*alpha  # 0.9
nu = 3.0
s = 1.0

t = betaprime.rvs(0.5, nu/2, size=n)
emp_vals = [mp.hyp2f1(1, alpha, alpha+beta, -s*nu*ti) for ti in t]
emp_mean = float(np.mean([float(v) for v in emp_vals]))

# Teori via dobbeltintegralet
B05 = mp.beta(0.5, nu/2)
norm_ab = mp.gamma(alpha+beta)/(mp.gamma(alpha)*mp.gamma(beta))

def I_of_x(x):
    f = lambda tt: (tt**(-0.5)) * ((1+tt)**(-(nu+1)/2)) / (1 + s*nu*x*tt)
    return (1/B05) * mp.quad(f, [0, mp.inf])

g = lambda x: (x**(alpha-1)) * ((1-x)**(beta-1)) * I_of_x(x)
theory_int = norm_ab * mp.quad(g, [0, 1])

print("Empirisk (s=1):", emp_mean)
print("Teori (integral):", theory_int)
print("Diff: ", emp_mean- theory_int)


## SAMPLES

In [31]:
path = "datasets/friedman/Friedman_N500_p10_sigma1.00_seed11.npz"
data = np.load(path)


In [32]:
import numpy as np

# ---------- 1) Simuler X ----------
def simulate_X(n, P, seed=42):
    rng = np.random.default_rng(seed)
    return rng.uniform(0.0, 1.0, size=(n, P))

# ---------- 2) Aktivering og derivert ----------
def get_activation(activation="tanh"):
    if activation == "tanh":
        phi = np.tanh
        def dphi(a): return 1.0 - np.tanh(a)**2
    elif activation == "relu":
        def phi(a): return np.maximum(0.0, a)
        def dphi(a): return (a > 0.0).astype(a.dtype)
    else:
        raise ValueError(f"Unsupported activation: {activation}")
    return phi, dphi

# ---------- 3) Hovedfunksjon: q for alle trekk ----------
def compute_q_for_fit(cmdstan_mcmc, N=1000, activation="tanh", seed=1, output_index=0, X=None):
    """
    Beregn q_{ell, j} for første-lagsvektene for hver trekk (draw).
    Returnerer:
      q_draws:  (n_draws, H, P)
      q_mean:   (H, P)  – gjennomsnitt over trekk
      X:        (N, P)  – datasettet brukt i beregningen
    """
    # Hent ut variabler fra Stan
    W1_all = cmdstan_mcmc.stan_variable("W_1")            # (draws, P, H)
    WL_all = cmdstan_mcmc.stan_variable("W_L")             # (draws, H, O)
    hb_all = cmdstan_mcmc.stan_variable("hidden_bias")     # (draws, L, H)
    sigma_all = cmdstan_mcmc.stan_variable("sigma")        # (draws,)
    Wint_all = cmdstan_mcmc.stan_variable("W_internal")    # (draws, max(L-1,1), H, H)

    draws, P, H = W1_all.shape
    O = WL_all.shape[2]
    L = hb_all.shape[1]

    if O == 0:
        raise ValueError("W_L has zero output nodes. Expected at least 1.")
    if output_index < 0 or output_index >= O:
        raise ValueError(f"output_index {output_index} out of range 0..{O-1}")

    if X is None:
        X = simulate_X(N, P, seed=seed)

    X_sq = X**2
    phi, dphi = get_activation(activation)

    q_draws = np.empty((draws, H, P), dtype=float)

    for s in range(draws):
        W1 = W1_all[s]            # (P, H)
        WL = WL_all[s]            # (H, O)
        hb = hb_all[s]            # (L, H)
        Wints = Wint_all[s]       # (max(L-1,1), H, H)
        sigma = float(sigma_all[s])

        # ----- Forward pass -----
        a_list = []
        h_list = []

        a = X @ W1 + hb[0]        # (N, H)
        h = phi(a)
        a_list.append(a); h_list.append(h)

        for l in range(1, L):
            Wl = Wints[l-1]       # (H, H)
            a = h @ Wl + hb[l]    # (N, H)
            h = phi(a)
            a_list.append(a); h_list.append(h)

        # ----- Backward: delta_L = d f / d a^(L) -----
        # lineær utgang: df/dh^(L) = WL[:, output_index]
        v = WL[:, output_index]           # (H,)
        delta = dphi(a_list[-1]) * v      # (N, H), broadcast over N

        # Bakover gjennom skjulte lag
        for l in range(L-2, -1, -1):
            Wnext = Wints[l]              # (H, H) – brukes bare hvis L>1
            delta = (delta @ Wnext.T) * dphi(a_list[l]) if L > 1 else delta

        delta1 = delta  # (N, H) == ∂f/∂a^(1)

        # ----- q: (1/sigma^2) * sum_i (delta1[i,ell]^2 * X[i,j]^2) -----
        D_sq = delta1**2                  # (N, H)
        Q = (X_sq.T @ D_sq) / (sigma**2)  # (P, H)
        q_draws[s] = Q.T                  # (H, P)

    q_mean = q_draws.mean(axis=0)         # (H, P)
    return q_draws, q_mean, X

# ---------- 4) Eksempelbruk ----------
# Velg riktig fit-objekt (CmdStanMCMC) fra dict-en din:
prior_q_dhs, prior_q_mean_dhs, X_train = compute_q_for_fit(
    prior_fits['Dirichlet Horseshoe']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_dhs, posterior_q_mean_dhs, X_train = compute_q_for_fit(
    posterior_N500_fits['Dirichlet Horseshoe tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

prior_q_dst, prior_q_mean_dst, X_train = compute_q_for_fit(
    prior_fits['Dirichlet Student T']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_dst, posterior_q_mean_dst, X_train = compute_q_for_fit(
    posterior_N500_fits['Dirichlet Student T tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

prior_q_rhs, prior_q_mean_rhs, X_train = compute_q_for_fit(
    prior_fits['Regularized Horseshoe']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_rhs, posterior_q_mean_rhs, X_train = compute_q_for_fit(
    posterior_N500_fits['Regularized Horseshoe tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

prior_q_gauss, prior_q_mean_gauss, X_train = compute_q_for_fit(
    prior_fits['Gaussian']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

posterior_q_gauss, posterior_q_mean_gauss, X_train = compute_q_for_fit(
    posterior_N500_fits['Gaussian tanh']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

prior_q_gamma, prior_q_mean_gamma, X_train = compute_q_for_fit(
    prior_fits['Dirichlet Gamma']['posterior'],
    N=1000,             
    activation='tanh',  
    seed=123,
    output_index=0,
    X = data["X_train"]      
)

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# ---- choose which node/input to inspect ----
node_idx = 1
input_idx = 1

## Dirichlet Horseshoe

In [6]:
# -------- PRIOR --------
prior_fit = prior_fits['Dirichlet Horseshoe']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]  # (draws_prior,)
phi_prior = prior_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]           # (draws_prior,)
q_prior   = prior_q_dhs[:, node_idx, input_idx]                               # (draws_prior,)

# -------- POSTERIOR --------
post_fit = posterior_N100_fits['Dirichlet Horseshoe tanh']['posterior']
tau_post = post_fit.stan_variable("tau")                                      # (draws_post,)
lam_post = post_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]    # (draws_post,)
phi_post = post_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]             # (draws_post,)
q_post   = posterior_q_dhs[:, node_idx, input_idx]                           # (draws_post,)

kappa_DHS_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2) * (phi_prior**2))
kappa_DHS_post = 1.0 / (1.0 + q_post * (tau_post**2) * (lam_post**2) * (phi_post**2))

## Dirichlet Student T

In [7]:
# -------- PRIOR --------
prior_fit = prior_fits['Dirichlet Student T']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]  # (draws_prior,)
phi_prior = prior_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]           # (draws_prior,)
q_prior   = prior_q_dhs[:, node_idx, input_idx]                               # (draws_prior,)

# -------- POSTERIOR --------
post_fit = posterior_N100_fits['Dirichlet Student T tanh']['posterior']
tau_post = post_fit.stan_variable("tau")                                      # (draws_post,)
lam_post = post_fit.stan_variable("lambda_tilde_data")[:, :, node_idx][:, input_idx]    # (draws_post,)
phi_post = post_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]             # (draws_post,)
q_post   = posterior_q_dhs[:, node_idx, input_idx]                           # (draws_post,)


kappa_DST_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2) * (phi_prior**2))
kappa_DST_post = 1.0 / (1.0 + q_post * (tau_post**2) * (lam_post**2) * (phi_post**2))

## Dirichlet Gamma

In [8]:
# -------- PRIOR --------
prior_fit = prior_fits['Dirichlet Gamma']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_data")[:, :, node_idx][:, input_idx]  # (draws_prior,)
phi_prior = prior_fit.stan_variable("phi_data")[:, :, node_idx][:, input_idx]           # (draws_prior,)
q_prior   = prior_q_gamma[:, node_idx, input_idx]                               # (draws_prior,)


kappa_DG_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2) * (phi_prior**2))

## Regularized Horseshoe

In [9]:
prior_fit = prior_fits['Regularized Horseshoe']['posterior']
tau_prior = prior_fit.stan_variable("tau")                                   # (draws_prior,)
lam_prior = prior_fit.stan_variable("lambda_tilde")[:, :, node_idx][:, input_idx]  # (draws_prior,)
q_prior   = prior_q_rhs[:, node_idx, input_idx]                               # (draws_prior,)

post_rhs_fit = posterior_N100_fits['Regularized Horseshoe tanh']['posterior']
tau_rhs_post = post_rhs_fit.stan_variable("tau")                                      # (draws_post,)
lam_rhs_post = post_rhs_fit.stan_variable("lambda_tilde")[:, :, node_idx][:, input_idx]    # (draws_post,)
q_rhs_post   = posterior_q_rhs[:, node_idx, input_idx]                           # (draws_post,)

kappa_HS_prior = 1.0 / (1.0 + q_prior * (tau_prior**2) * (lam_prior**2))
kappa_HS_post = 1.0 / (1.0 + q_rhs_post * (tau_rhs_post**2) * (lam_rhs_post**2))

## Gaussian

In [10]:
q_prior   = prior_q_gauss[:, node_idx, input_idx]                               # (draws_prior,)

q_gauss_post   = posterior_q_gauss[:, node_idx, input_idx]                           # (draws_post,)

kappa_gauss_prior = 1.0 / (1.0 + q_prior)
kappa_gauss_post = 1.0 / (1.0 + q_gauss_post)

In [ ]:
# -------- Plot: prior vs posterior overlays --------
bins = np.linspace(0.0, 1.0, 40)  # common bins so densities are comparable

fig, axes = plt.subplots(2, 2, figsize=(8, 6), sharex=True, sharey=True)


# Horseshoe
axes[0, 0].hist(kappa_gauss_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[0, 0].hist(kappa_gauss_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[0, 0].set_title("Gaussian")
axes[0, 0].set_xlabel(r"$\kappa_{\ell j}$")
axes[0, 0].legend()

# Horseshoe
axes[0, 1].hist(kappa_HS_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[0, 1].hist(kappa_HS_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[0, 1].set_title("RHS")
axes[0, 1].set_xlabel(r"$\kappa_{\ell j}$")
axes[0, 1].legend()

# Dirichlet–Horseshoe
axes[1, 0].hist(kappa_DHS_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[1, 0].hist(kappa_DHS_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[1, 0].set_title("DHS")
axes[1, 0].set_xlabel(r"$\kappa_{\ell j}$")
axes[1, 0].set_ylabel("Density")
axes[1, 0].legend()

# Dirichlet–Horseshoe
axes[1, 1].hist(kappa_DST_prior, bins=bins, density=True, alpha=0.45, label="Prior")
axes[1, 1].hist(kappa_DST_post,  bins=bins, density=True, alpha=0.45, label="Posterior")
axes[1, 1].set_title("DS-T")
axes[1, 1].set_xlabel(r"$\kappa_{\ell j}$")
axes[1, 1].legend()


plt.suptitle(f"Node {node_idx}, input {input_idx}")#: Prior vs Posterior")
plt.tight_layout()
plt.show()

In [ ]:
plt.plot()
plt.hist(kappa_DG_prior, bins=50, density=True, alpha=0.45, label="Dirichlet Gamma prior")
plt.legend()
#plt.xlim(3, 50)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.figure(figsize=(7,5))

colors = {
    "Gaussian": "C0",
    "RHS": "C1",
    "DHS": "C2",
    "DS-T": "C3",
}

# Plot linjene
sns.kdeplot(kappa_gauss_prior, linestyle="--", color=colors["Gaussian"])
sns.kdeplot(kappa_gauss_post, linestyle="-",  color=colors["Gaussian"])
sns.kdeplot(kappa_HS_prior, linestyle="--", color=colors["RHS"])
sns.kdeplot(kappa_HS_post, linestyle="-",  color=colors["RHS"])
sns.kdeplot(kappa_DHS_prior, linestyle="--", color=colors["DHS"])
sns.kdeplot(kappa_DHS_post, linestyle="-",  color=colors["DHS"])
sns.kdeplot(kappa_DST_prior, linestyle="--", color=colors["DS-T"])
sns.kdeplot(kappa_DST_post, linestyle="-",  color=colors["DS-T"])

plt.xlabel(r"$\kappa_{\ell j}$")
plt.ylabel("Density")
plt.title("Prior shrinkage factors – KDE")

# --- Lag legend ---
# Farger (modeller)
model_handles = [Line2D([0], [0], color=color, lw=2, label=model) 
                 for model, color in colors.items()]

# Linjestil (prior/posterior)
style_handles = [
    Line2D([0], [0], color="black", linestyle="--", lw=2, label="Prior"),
    Line2D([0], [0], color="black", linestyle="-",  lw=2, label="Posterior")
]

# Kombiner
handles = model_handles + style_handles
labels = [h.get_label() for h in handles]

plt.legend(handles=handles, labels=labels, title="Model / Distribution", ncol=1)
plt.show()


## Regularized versions

For the Regularized Horseshoe, when $c<\infty$. the shrinkage profile becomes approximately equivalent to that of the horseshoe shifted from the interval $(0, 1)$ to $(b_{\ell, j}, 1)$ where $b_{\ell, j}=\frac{1}{q_{\ell, j}\tau{\ell}^2c^2}$. The shrinkage factor then approximately satisfies $\tilde{\kappa}_{\ell, j}=(1-b_j)\kappa_{\ell, j} + b_j$ where $\tilde{\kappa}_{\ell, j}$ is the shrinkage factor of the original horseshoe. From this we get $1-\tilde{\kappa}_{\ell, j}=(1-b_j)(1-\kappa_{\ell, j})$. Assuming roughly the same scale of inputs, then $b_{\ell, j}=b_{\ell}$ and the effective model complexity becomes $\tilde{m}_{\ell, eff}=(1-b_{\ell}) m_{\ell, eff}$ where $m_{\ell, eff}=\sum_{j=1}^p(1-\kappa_{\ell, j})$ denotes the effective number of nonzero weights into node $l$ (I THINK)

In [33]:
import numpy as np

def compute_kappa(fit, q, node_idx, input_idx, model_type='gaussian'):
    """
    Returnerer arrays per trekning (S,): kappa_original, b_j, kappa_tilde
    """
    q_hp = np.asarray(q)[:, node_idx, input_idx]  # (S,)

    if model_type == 'gaussian':
        tau = 1.0
        lam = 1.0
        c_sq = 1.0
        phi = 1.0
    else:
        tau = fit.stan_variable("tau")                          # (S,)
        c_sq = fit.stan_variable("c_sq")[:, node_idx]           # (S,)

        if model_type == 'rhs':
            lam = fit.stan_variable("lambda_tilde")[:, node_idx, input_idx]       # (S,)
            phi = 1.0
        elif model_type == 'dhs' or 'dst':
            lam = fit.stan_variable("lambda_tilde_data")[:, node_idx, input_idx]  # (S,)
            phi = fit.stan_variable("phi_data")[:, node_idx, input_idx]           # (S,)
        # elif model_type == 'dst':
        #     lam = fit.stan_variable("lambda_tilde_data")[:, node_idx, input_idx]       # (S,)
        #     phi = fit.stan_variable("phi_data")[:, node_idx, input_idx]           # (S,)
        else:
            raise ValueError("model_type må være 'gaussian', 'rhs', 'dhs' eller 'dst'.")

    kappa_original = 1.0 / (1.0 + q_hp * (tau**2) * (lam**2) * (phi))  # (S,)
    b_j            = 1.0 / (1.0 + q_hp * (tau**2) * c_sq * (phi))      # (S,)
    kappa_tilde    = (1.0 - b_j) * kappa_original + b_j                # (S,)

    return kappa_original, b_j, kappa_tilde


def compute_kappa_per_input(fit, q, node_idx, model_type='gaussian'):
    """
    For en gitt node: beregn per input p
      - E[1 - kappa_tilde]_p
      - E[(1 - kappa)(1 - b_j)]_p
    og returnér begge som (P,) + totalsum som skalarer.
    """
    S, H, P = np.asarray(q).shape
    mean_1_minus_kappa_tilde = np.zeros(P)
    mean_prod_identity       = np.zeros(P)

    for p in range(P):
        kappa, b_j, kappa_tilde = compute_kappa(fit, q, node_idx=node_idx, input_idx=p, model_type=model_type)

        # Riktig aggregering: gjennomsnitt av uttrykket per trekning
        mean_1_minus_kappa_tilde[p] = np.mean(1.0 - kappa_tilde)
        mean_prod_identity[p]       = np.mean((1.0 - kappa) * (1.0 - b_j))

    # Summér over inputs (antall ikke-null-vekter inn i noden i forventning)
    total_meff_tilde = np.sum(mean_1_minus_kappa_tilde)
    total_meff_check = np.sum(mean_prod_identity)

    return total_meff_tilde, total_meff_check


In [ ]:
nodes = 16
gauss_fit = prior_fits['Gaussian']['posterior']
rhs_fit = prior_fits['Regularized Horseshoe']['posterior']
dhs_fit = prior_fits['Dirichlet Horseshoe']['posterior']
dst_fit = prior_fits['Dirichlet Student T']['posterior']

nonzero = np.zeros((4, nodes))

for i in range(nodes):
    meff_tilde_g, _ = compute_kappa_per_input(
        gauss_fit, prior_q_gauss, node_idx=i, model_type='gaussian'
    )
    meff_tilde_rhs, _ = compute_kappa_per_input(
        rhs_fit, prior_q_rhs, node_idx=i, model_type='rhs'
    )
    meff_tilde_dhs, _ = compute_kappa_per_input(
        dhs_fit, prior_q_dhs, node_idx=i, model_type='dhs'
    )
    meff_tilde_dst, _ = compute_kappa_per_input(
        dst_fit, prior_q_dst, node_idx=i, model_type='dst'
    )
    nonzero[0, i] = meff_tilde_g
    nonzero[1, i] = meff_tilde_rhs
    nonzero[2, i] = meff_tilde_dhs
    nonzero[3, i] = meff_tilde_dst
    
print("Prior number of nonzero weights: ", np.sum(nonzero, axis=1))
print("Prior percentage of nonzero weights: ", np.sum(nonzero, axis=1)/128)

In [ ]:
nodes = 16
gauss_fit = posterior_N500_fits['Gaussian tanh']['posterior']
rhs_fit = posterior_N500_fits['Regularized Horseshoe tanh']['posterior']
dhs_fit = posterior_N500_fits['Dirichlet Horseshoe tanh']['posterior']
dst_fit = posterior_N500_fits['Dirichlet Student T tanh']['posterior']

nonzero = np.zeros((4, nodes))

for i in range(nodes):
    meff_tilde_g, _ = compute_kappa_per_input(
        gauss_fit, posterior_q_gauss, node_idx=i, model_type='gaussian'
    )
    meff_tilde_rhs, _ = compute_kappa_per_input(
        rhs_fit, posterior_q_rhs, node_idx=i, model_type='rhs'
    )
    meff_tilde_dhs, _ = compute_kappa_per_input(
        dhs_fit, posterior_q_dhs, node_idx=i, model_type='dhs'
    )
    meff_tilde_dst, _ = compute_kappa_per_input(
        dst_fit, posterior_q_dst, node_idx=i, model_type='dst'
    )
    nonzero[0, i] = meff_tilde_g
    nonzero[1, i] = meff_tilde_rhs
    nonzero[2, i] = meff_tilde_dhs
    nonzero[3, i] = meff_tilde_dst
    
print("Posterior number of nonzero weights: ", np.sum(nonzero, axis=1))
print("Posterior percentage of nonzero weights: ", np.sum(nonzero, axis=1)/128)

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
path = "datasets/friedman/Friedman_N100_p10_sigma1.00_seed1.npz"
data = np.load(path)
x_train = data["X_train"]

fits = prior_fits

P = 10
H = 16
L = 1
out_nodes = 1

layer_structure = {
    'input_to_hidden': {'name': 'W_1', 'shape': (P, H)},
    'hidden_to_output': {'name': 'W_L', 'shape': (H, out_nodes)}
}


def build_single_draw_weights(fits, layer_structure, draw_idx):
    """Return {model: {'W_1': (P,H), 'W_L': (H,O)}} for ONE draw."""
    out = {}
    for name, fd in fits.items():
        fit = fd["posterior"]
        W1 = fit.stan_variable(layer_structure['input_to_hidden']['name'])[draw_idx]
        WL = fit.stan_variable(layer_structure['hidden_to_output']['name'])[draw_idx]
        WL = WL.reshape(layer_structure['hidden_to_output']['shape'])
        out[name] = {"W_1": W1, "W_L": WL}
    return out

def scale_W1_for_plot(model_means, mode='global'):
    """
    Skalerer alle W_1 til [-1, 1] for rettferdig sammenligning av edge-tykkelser.

    mode:
      - 'global' : én felles skala over alle modeller (mest sammenlignbar)
      - 'per_model': egen skala per modell (uavhengig sammenligning)
      - 'per_node' : skalerer hver kolonne (node) separat til [-1,1]

    Returnerer: scaled_model_means (samme struktur som input), scale_info
    """
    scaled = {}
    if mode == 'global':
        gmax = max(np.abs(m['W_1']).max() for m in model_means.values())
        gmax = max(gmax, 1e-12)
        for name, m in model_means.items():
            W1s = m['W_1'] / gmax
            out = {k: v for k, v in m.items()}
            out['W_1'] = W1s
            scaled[name] = out
        return scaled, {'mode': 'global', 'scale': gmax}

    elif mode == 'per_model':
        for name, m in model_means.items():
            s = max(np.abs(m['W_1']).max(), 1e-12)
            out = {k: v for k, v in m.items()}
            out['W_1'] = m['W_1'] / s
            scaled[name] = out
        return scaled, {'mode': 'per_model'}

    elif mode == 'per_node':
        for name, m in model_means.items():
            W1 = m['W_1'].copy()
            P, H = W1.shape
            for h in range(H):
                colmax = max(np.abs(W1[:, h]).max(), 1e-12)
                W1[:, h] = W1[:, h] / colmax
            out = {k: v for k, v in m.items()}
            out['W_1'] = W1
            scaled[name] = out
        return scaled, {'mode': 'per_node'}

    else:
        raise ValueError("mode must be 'global', 'per_model', or 'per_node'")

def plot_models_with_activations(model_means, layer_sizes,
                                 activations=None, activation_color_max=None,
                                 ncols=3, figsize_per_plot=(5,4), signed_colors=False):
    """
    model_means: dict {model_name: {'W_1':(P,H), 'W_L':(H,O), optional 'W_internal':[...]} }
    layer_sizes: f.eks [P, H, O] eller [P, H, H, O] ved internlag
    activations: dict {model_name: (H,)} – aktiveringsfrekvens kun for første skjulte lag
    activation_color_max: global maks for skalering av farger (hvis None brukes 1.0)
    """
    names = list(model_means.keys())
    n_models = len(names)
    nrows = int(np.ceil(n_models / ncols))
    figsize = (figsize_per_plot[0] * ncols, figsize_per_plot[1] * nrows)

    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    if nrows * ncols == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    # Skru av blanke akser
    for ax in axes[n_models:]:
        ax.axis('off')

    for ax, name in zip(axes, names):
        weights = model_means[name]
        G = nx.DiGraph()
        pos, nodes_per_layer, node_colors = {}, [], []

        # Noder med posisjon og farge
        for li, size in enumerate(layer_sizes):
            ids = []
            ycoords = np.linspace(size - 1, 0, size) - (size - 1) / 2
            for i in range(size):
                nid = f"L{li}_{i}"
                G.add_node(nid)
                pos[nid] = (li, ycoords[i])
                ids.append(nid)

                if activations is not None and li == 1:  # kun første skjulte lag
                    #a = activations.get(name, np.zeros(size))
                    a = activations.get(name, np.zeros(size))
                    a = np.asarray(a).ravel()   # <-- flater til 1D array
                    scale = activation_color_max if activation_color_max is not None else 1.0
                    val = float(np.clip(a[i] / max(scale, 1e-12), 0.0, 1.0))
                    color = plt.cm.winter(val)
                else:
                    color = 'lightgray'
                node_colors.append(color)

            nodes_per_layer.append(ids)

        edge_colors, edge_widths = [], []

        def add_edges(W, inn, ut):
            for j, out_n in enumerate(ut):
                for i, in_n in enumerate(inn):
                    w = float(W[i, j])
                    G.add_edge(in_n, out_n, weight=abs(w))
                    edge_colors.append('red' if w >= 0 else 'blue')
                    edge_widths.append(abs(w))

        # input -> hidden(1)
        add_edges(weights['W_1'], nodes_per_layer[0], nodes_per_layer[1])

        # ev. internlag
        if 'W_internal' in weights:
            for l, Win in enumerate(weights['W_internal']):
                add_edges(Win, nodes_per_layer[l+1], nodes_per_layer[l+2])

        # siste hidden -> output
        add_edges(weights['W_L'], nodes_per_layer[-2], nodes_per_layer[-1])

        nx.draw(G, pos, ax=ax,
                node_color=node_colors,
                edge_color=(edge_colors if signed_colors else 'red'),
                width=[G[u][v]['weight'] for u,v in G.edges()],
                with_labels=False, node_size=400, arrows=False)

        ax.set_title(name, fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    return fig

def compute_hidden_activation(fit_dict, x_train, draw_idx):
    fit = fit_dict['posterior']
    W1 = fit.stan_variable('W_1')[draw_idx, :, :]          # (P,H)
    try:
        b1 = fit.stan_variable('hidden_bias')[draw_idx, :] # (H,)
    except Exception:
        b1 = np.zeros(W1.shape[1])
    # tanh i [-1,1]
    a_full = np.tanh(x_train @ W1 + b1)             # (H,)
    a=np.mean(a_full, axis=0)
    return a


In [ ]:
# Velg en observasjon å "lyse opp" nodefargene med
obs_idx = 3
draw_idx = 69 #pick_draw_idx(prior_fits, seed=42)      # one common draw across models
prior_draws = build_single_draw_weights(prior_fits, layer_structure, draw_idx)

# 1) Beregn aktivasjoner for ALLE modellene
activations = {}
for name, fd in prior_fits.items():
    a = compute_hidden_activation(fd, x_train, draw_idx)
    activations[name] = np.abs(a)      

# 2) Skaler vekter for plotting (som før)
scaled, _ = scale_W1_for_plot(prior_draws, mode='per_model')

# 3) Kall plottet med aktivasjoner
# Siden tanh ∈ [-1,1] og vi bruker |a|, så sett activation_color_max=1.0
fig = plot_models_with_activations(
    scaled,
    layer_sizes=[P, H, out_nodes],
    activations=activations,
    activation_color_max=1.0,
    ncols=2,
)
plt.show()

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


# 2) Bygg prunede posterior-mean vekter for alle modeller (pruner K minste |W_1|)
def build_pruned_means(fits, layer_structure, sparsity_level):
    """
    fits: dict {model_name: {"posterior": CmdStanMCMC}}
    layer_structure: {'input_to_hidden': {'name':'W_1','shape':(P,H)},
                      'hidden_to_output':{'name':'W_L','shape':(H,O)}}
                      # optional: 'hidden_to_hidden': {'name':..., 'shape': [(H,H), ...]}
    sparsity_level: andel av W_1-elementer som settes til 0 (0.0–1.0)

    Returnerer dict {model_name: {'W_1':(P,H), 'W_L':(H,O), 'W_internal':[...](optional)}}
    """
    model_means = {}

    for name, fd in fits.items():
        fit = fd["posterior"]

        # Posterior means
        W1_name = layer_structure['input_to_hidden']['name']
        WL_name = layer_structure['hidden_to_output']['name']

        W1 = fit.stan_variable(W1_name).mean(axis=0)  # (P,H)
        WL = fit.stan_variable(WL_name).mean(axis=0)  # (H,O)
        WL = WL.reshape(layer_structure['hidden_to_output']['shape'])

        means = {'W_1': W1.copy(), 'W_L': WL}

        # Optional internlag
        if 'hidden_to_hidden' in layer_structure:
            HH_name = layer_structure['hidden_to_hidden']['name']
            HH_shapes = layer_structure['hidden_to_hidden']['shape']  # list of (H,H)
            raw = fit.stan_variable(HH_name).mean(axis=0)
            means['W_internal'] = [raw[i].reshape(HH_shapes[i]) for i in range(len(HH_shapes))]

        # Pruning av W_1 (beholder de største |w|). Samme logikk som i koden din.
        P, H = means['W_1'].shape
        K = int(np.floor(sparsity_level * P * H))
        if K > 0:
            flat_abs = np.abs(means['W_1'].ravel())
            idx_prune = np.argpartition(flat_abs, K)[:K]   # indeksene til de K minste
            mask = np.ones_like(flat_abs, dtype=float)
            mask[idx_prune] = 0.0
            means['W_1'] = (means['W_1'].ravel() * mask).reshape(P, H)

        model_means[name] = means

    return model_means

def compute_hidden_activation_from_means(model_means, x_train):
    """
    model_means: dict {model_name: {'W_1':(P,H), 'W_L':(H,O)}}
    x_train: (N,P)

    Returnerer dict {model_name: (H,)} med gj.snitt aktivasjon pr. node.
    """
    activations = {}
    for name, m in model_means.items():
        W1 = m['W_1']                 # (P,H)
        b1 = m.get('hidden_bias', np.zeros(W1.shape[1]))  # hvis bias finnes, ellers null
        a_full = np.tanh(x_train @ W1 + b1)   # (N,H)
        activations[name] = np.abs(a_full).mean(axis=0)  # (H,)
    return activations

# def input_contribution_shares_W1(fit, x_train, weight_by_activity=True):
#     """
#     Returnerer (P,H) med s_{i,h} ≥ 0 og sum_i s_{i,h} ≈ 1.
#     s_{i,h} lages ved å:
#       1) for hvert datapunkt n, beregne |x_{n,i} w_{i,h}| og normalisere over i,
#       2) gjennomsnittlig (valgfritt) vektet med |tanh(pre_{n,h})| over n.

#     weight_by_activity=True anbefales for tanh.
#     """
#     W1 = fit['posterior'].stan_variable('W_1').mean(axis=0)   # (P,H)
#     try:
#         b1 = fit['posterior'].stan_variable('hidden_bias').mean(axis=0)
#     except Exception:
#         b1 = np.zeros(W1.shape[1])

#     pre = x_train @ W1 + b1                                   # (N,H)
#     act = np.abs(np.tanh(pre))                                # (N,H)

#     N, H = pre.shape
#     P = W1.shape[0]
#     shares = np.zeros((P, H))

#     absW = np.abs(W1)                   # (P,H)
#     absX = np.abs(x_train)              # (N,P)

#     eps = 1e-12
#     for h in range(H):
#         contrib = absX * absW[:, h].reshape(1, P)             # (N,P): |x|*|w|
#         denom = np.clip(contrib.sum(axis=1, keepdims=True), eps, None)  # (N,1)
#         s_n = contrib / denom                                  # (N,P), radvis sum=1

#         if weight_by_activity:
#             w = act[:, h]                                      # (N,)
#             wsum = float(w.sum())
#             if wsum < eps:
#                 shares[:, h] = 1.0 / P
#             else:
#                 shares[:, h] = (s_n.T @ w) / wsum              # (P,)
#         else:
#             shares[:, h] = s_n.mean(axis=0)                    # uvektet snitt

#     return shares

# def summarize_active_weights(shares, thresholds=(0.5, 0.8, 0.9)):
#     """
#     shares: (P,H) med s_{i,h} ≥ 0 og kolonnevis sum ~ 1.
#     thresholds: iterable, f.eks (0.5, 0.8, 0.9)

#     Returnerer:
#       - k_for_threshold: dict {thr: array(H,)}  # minste k per node
#       - neff: array(H,)
#       - gini: array(H,)
#     """
#     P, H = shares.shape
#     k_for_threshold = {thr: np.zeros(H, dtype=int) for thr in thresholds}
#     neff = np.zeros(H)
#     gini = np.zeros(H)

#     for h in range(H):
#         v = np.sort(shares[:, h])[::-1]               # synkende
#         csum = np.cumsum(v)
#         total = csum[-1] if csum.size else 0.0
#         total = max(total, 1e-12)

#         for thr in thresholds:
#             k = int(np.searchsorted(csum, thr * total) + 1)  # +1 -> antall elementer
#             k_for_threshold[thr][h] = min(k, P)

#         # N_eff = 1 / sum s_i^2
#         neff[h] = 1.0 / max((shares[:, h] ** 2).sum(), 1e-12)

#         # Gini (diskret): 1 - 2 * (sum_{i=1}^P (P+1-i) s_{(i)}) / (P * sum s_i)
#         w = v
#         gini[h] = 1.0 - 2.0 * ( (np.arange(1, P+1) * w).sum() ) / (P * total)

#     return k_for_threshold, neff, gini



In [ ]:
path = "datasets/friedman/Friedman_N500_p10_sigma1.00_seed11.npz"
data = np.load(path)
x_train = data["X_train"]

fits = posterior_N500_fits       # posterior_N100_fits

P = 10
H = 16
L = 1
out_nodes = 1

layer_structure = {
    'input_to_hidden': {'name': 'W_1', 'shape': (P, H)},
    'hidden_to_output': {'name': 'W_L', 'shape': (H, out_nodes)}
}

# 1) bygg/prune posterior-mean som før (fra tidligere funksjoner dine)
pruned_model_means = build_pruned_means(fits, layer_structure, sparsity_level=0.0)

# 2) skaler W1 for sammenlignbar tykkelse
scaled_means, scale_info = scale_W1_for_plot(pruned_model_means, mode='per_model')  # eller 'per_model' / 'per_node'
activations = compute_hidden_activation_from_means(pruned_model_means, x_train)
all_vals = np.concatenate(list(activations.values()))
vmin, vmax = all_vals.min(), all_vals.max()

# 4) plot med skalerte vekter (W_1 byttes ut)
fig = plot_models_with_activations(
    scaled_means,
    layer_sizes=[P, H, 1],
    activations=activations,
    activation_color_max=vmax,
    ncols=2,
    signed_colors=False
)


In [68]:
import numpy as np
from itertools import combinations

def mean_abs(arr):  # arr: (S, ...)
    return np.mean(np.abs(np.asarray(arr)), axis=0)

def nid_single_hidden(posterior, W1_name="W_1", WL_name_candidates=("W_L","W_2")):
    """
    posterior: CmdStanMCMC-objekt
    W_1: shape (S, P, H)  (input -> hidden), som du har
    W_L: shape (S, H) eller (S, H, O)  (hidden -> output)
    """
    W1_samps = posterior.stan_variable(W1_name)          # (S, P, H)
    # Finn navn for siste lag
    for nm in WL_name_candidates:
        try:
            WL_samps = posterior.stan_variable(nm)       # (S, H) eller (S, H, O)
            break
        except Exception:
            WL_samps = None
    if WL_samps is None:
        raise ValueError("Fant ikke siste-lag-vekter (prøv å angi riktig navn i WL_name_candidates).")

    # Posterior plug-in: gjennomsnitt av absoluttverdier
    W1_abs = mean_abs(W1_samps)                          # (P, H)
    WL_abs = mean_abs(WL_samps)                          # (H,) eller (H, O)

    # z^(1): aggregert node-innflytelse (sum over outputs hvis flere)
    if WL_abs.ndim == 1:
        z1 = WL_abs                                      # (H,)
    else:
        z1 = WL_abs.sum(axis=1)                          # (H,)

    P, H = W1_abs.shape

    # Main effects: ω({j}) = Σ_i z_i * |W1[j,i]|
    omega_main = (W1_abs * z1[None, :]).sum(axis=1)      # (P,)

    # Pairwise: ω({j,k}) = Σ_i z_i * min(|W1[j,i]|, |W1[k,i]|)
    omega_pair = np.zeros((P, P))
    for j, k in combinations(range(P), 2):
        mins = np.minimum(W1_abs[j, :], W1_abs[k, :])    # (H,)
        omega = np.dot(z1, mins)                         # skalar
        omega_pair[j, k] = omega_pair[k, j] = omega

    return z1, omega_main, omega_pair


In [ ]:
post = posterior_N100_fits['Dirichlet Student T tanh']['posterior'] 
z1, omega_main, omega_pair = nid_single_hidden(post) # W_1=(S,P,H), W_L/(W_2)=(S,H[,O]) # Eksempler: # - topp 10 viktigste noder etter z1: 
top_nodes = np.argsort(-z1) # - topp 10 viktigste features (main effects): 
top_feats = np.argsort(-omega_main) # - sterkeste parvise interaksjoner: 
P = omega_pair.shape[0] 
pairs = [(j, k, omega_pair[j, k]) for j in range(P) for k in range(j+1, P)] 
top_pairs = sorted(pairs, key=lambda t: -t[2])[:10]

res = np.array(omega_main/(np.sum(omega_main)))
print(np.round(res, 3))

In [ ]:
print("\nTop 5 feature pairs (interactions):")
for (j,k,score) in top_pairs[:10]:
    print(f" Pair ({j},{k}): {score:.3f}")


In [ ]:
import numpy as np

def gini_coefficient(x):
    """
    Gini-indeks for en vektor x (ikke-negativ).
    Hvis alle elementer like → 0, hvis konsentrert på få elementer → nær 1.
    """
    x = np.asarray(x, dtype=float).flatten()
    if np.all(x == 0):
        return 0.0
    x = np.sort(np.abs(x))   # bruker absoluttverdier av vektene
    n = len(x)
    cumx = np.cumsum(x)
    return (n + 1 - 2*np.sum(cumx)/cumx[-1]) / n

def node_weight_stats(W1_samples, node_idx):
    """
    W1_samples: (S, P, H)
    node_idx: valgt skjult node
    Returnerer:
      ginis: (S,) gini-indeks per sample
      sums:  (S,) sum av abs(vekter) per sample
    """
    ginis = []
    for s in range(W1_samples.shape[0]):
        w = W1_samples[s, :, node_idx]   # (P,)
        ginis.append(gini_coefficient(w))
    return np.array(ginis)

# Gaussian
W1_gauss = posterior_N500_fits['Gaussian tanh']['posterior'].stan_variable("W_1")  # (S,P,H)
ginis_g = node_weight_stats(W1_gauss, node_idx=2)

# RHS
W1_gauss = posterior_N500_fits['Regularized Horseshoe tanh']['posterior'].stan_variable("W_1")  # (S,P,H)
ginis_rhs = node_weight_stats(W1_gauss, node_idx=2)

# DHS
W1_dhs = posterior_N500_fits['Dirichlet Horseshoe tanh']['posterior'].stan_variable("W_1")
ginis_dhs = node_weight_stats(W1_dhs, node_idx=2)

# DS-T
W1_dhs = posterior_N500_fits['Dirichlet Student T tanh']['posterior'].stan_variable("W_1")
ginis_dst = node_weight_stats(W1_dhs, node_idx=2)

print("Gaussian node 1:")
print(f"  Gini mean={ginis_g.mean():.3f}, q05={np.quantile(ginis_g,0.05):.3f}, q95={np.quantile(ginis_g,0.95):.3f}")

print("RHS node 1:")
print(f"  Gini mean={ginis_rhs.mean():.3f}, q05={np.quantile(ginis_rhs,0.05):.3f}, q95={np.quantile(ginis_rhs,0.95):.3f}")

print("DHS node 1:")
print(f"  Gini mean={ginis_dhs.mean():.3f}, q05={np.quantile(ginis_dhs,0.05):.3f}, q95={np.quantile(ginis_dhs,0.95):.3f}")

print("DS-T node 1:")
print(f"  Gini mean={ginis_dst.mean():.3f}, q05={np.quantile(ginis_dst,0.05):.3f}, q95={np.quantile(ginis_dst,0.95):.3f}")


## NODES, maybe not as important.

In [ ]:
from utils.visualize_networks import compute_activation_frequency, extract_all_pruned_means, plot_all_networks_subplots_activations
path = "datasets/friedman/Friedman_N100_p10_sigma1.00_seed1.npz"
data = np.load(path)
x_train = data["X_train"]

node_activation_colors = {
    model_name: compute_activation_frequency(fit, x_train)
    for model_name, fit in posterior_N100_fits.items()
}
layer_sizes = [P] + [H]*L + [out_nodes]
# Flatten and find the global maximum
all_freqs = np.concatenate(list(node_activation_colors.values()))
global_max = all_freqs.max()
print(global_max)

In [ ]:
from utils.visualize_networks import extract_all_pruned_node_means, plot_all_networks_subplots_activations
num_nodes_to_prune = 14  # for example
pruned_model_means_nodes = extract_all_pruned_node_means(posterior_N100_fits, layer_structure, num_nodes_to_prune)

p_nodes, widths_nodes = plot_all_networks_subplots_activations(
    pruned_model_means_nodes, layer_sizes, node_activation_colors,
    activation_color_max=global_max, signed_colors=False
)
